In [1]:
import numpy as np
from scipy.linalg import null_space
from scipy.linalg import solve, LinAlgError
from cvxopt import matrix, solvers
from scipy.optimize import minimize

In [2]:
def is_positive_definite(A):
    # Check if all eigenvalues are positive
    eigenvalues = np.linalg.eigvals(A)
    return np.all(eigenvalues > 0)

def is_positive_semidefinite(A):
    # Check if all eigenvalues are non-negative
    eigenvalues = np.linalg.eigvals(A)
    return np.all(eigenvalues >= 0)

def is_singular(A):
    # Check if determinant is zero
    return np.linalg.det(A) == 0

def spectral_norm(A):
    # Compute the singular values of the matrix
    singular_values = np.linalg.svd(A, compute_uv=False)
    # The spectral norm is the largest singular value
    return np.max(singular_values)

def is_y_valid(A, y):
    return np.linalg.norm(y) >= spectral_norm(A)

def get_initial_W(A, x, b):
    W = np.where((A @ x).T >= b)[1].tolist()
    return W

def calculate_pk(G, g_k, A_w):
    if A_w.size == 0:
        # If no active constraints, solve G * p_k = -g_k
        return -np.linalg.solve(G, g_k)
    else:
        # Setup the KKT system
        zeros_block = np.zeros((A_w.shape[0], A_w.shape[0]))
        KKT_matrix = np.block([
            [G, A_w.T],
            [A_w, zeros_block]
        ])
        zeros_rhs = np.zeros(A_w.shape[0])
        rhs = np.concatenate([-g_k, zeros_rhs])
        
        # Solve the KKT system
        solution = np.linalg.solve(KKT_matrix, rhs)
        p_k = solution[:G.shape[0]]  # Extract p_k from the solution
        return p_k

def calc_alpha_k(A, W, b, p_k, x):
    alpha_k = 1
    blocking_constraint_index = None
    print("Now inside alpha calculation:")
    for i in range(A.shape[0]):
        if i not in W and np.dot(A[i], p_k) < 0:
            print(f"    Constraint {i}:")
            temp_nom = b[i] - np.dot(A[i], x)
            print(f"    Nominator b[{i}] - A[{i}] @ x = {b[i]} - {np.dot(A[i], x)} = {temp_nom}")
            temp_denom = np.dot(A[i], p_k)
            print(f"    Denominator A[{i}] @ p_k = {temp_denom}")
            temp_alpha = temp_nom / temp_denom
            if temp_alpha < alpha_k:
                alpha_k = temp_alpha
                blocking_constraint_index = i
    # print(f"alpha_k:{alpha_k}")
    return alpha_k, blocking_constraint_index

def cvxopt_sol(Q, c, A, b):
    P = matrix(Q)
    q = matrix(c)
    G = matrix(A)
    h = matrix(b)
    solution = solvers.qp(P, q, G, h)
    return x

In [3]:
M = np.array([[1, 1]])
G = np.dot(M.T, M)
G_tilde = np.zeros((4, 4))  # shape is n*2 x n*2
G_tilde[:2, :2] = G  # position would be at [:n, :n]

y = np.array([2])
c = -np.dot(M.T, y)
c_tilde = np.zeros((4, 1))  # shape is n*2 x 1
c_tilde[:2] = c.reshape(-1, 1) # position would be at [:n]

x = np.array([[0.1, 1]])
z = np.abs(x)
z_tilde = np.vstack([x.T, z.T])

In [4]:
print(f"Shape of original matrix M: {M.shape}")
print(M)
print(f"Shape of G = M.T @ M: {G.shape}")
print(G)
print(f"Shape of y: {y.shape}")
print(y)
print(f"Shape of c: {c.shape}")
print(c)

print(f"\nIs the initialization of y valid? {is_y_valid(M, y)}")
print(f"Matrix G is singular? {is_singular(G)}")
print(f"Matrix G is positive definite? {is_positive_definite(G)}")
print(f"Matrix G is positive semidefinite? {is_positive_semidefinite(G)}")

print(f"\nShape of initial guess for x: {x.shape}")
print(x)
print(f"Shape of introduced variable z (abs(x)): {z.shape}")
print(z)

print("Shapes of reformulated problem:")
print(f"Shape of c_tilde (c & 0-vector concatonated): {c_tilde.shape}")
print(c_tilde)
print(f"Shape of z_tilde (x & z concatonated): {z_tilde.shape}")
print(z_tilde)

Shape of original matrix M: (1, 2)
[[1 1]]
Shape of G = M.T @ M: (2, 2)
[[1 1]
 [1 1]]
Shape of y: (1,)
[2]
Shape of c: (2,)
[-2 -2]

Is the initialization of y valid? True
Matrix G is singular? True
Matrix G is positive definite? False
Matrix G is positive semidefinite? True

Shape of initial guess for x: (1, 2)
[[0.1 1. ]]
Shape of introduced variable z (abs(x)): (1, 2)
[[0.1 1. ]]
Shapes of reformulated problem:
Shape of c_tilde (c & 0-vector concatonated): (4, 1)
[[-2.]
 [-2.]
 [ 0.]
 [ 0.]]
Shape of z_tilde (x & z concatonated): (4, 1)
[[0.1]
 [1. ]
 [0.1]
 [1. ]]


In [5]:
constraint_1 = np.hstack([-np.eye(2), np.eye(2)])               # -x + u >= 0
constraint_2 = np.hstack([np.eye(2), np.eye(2)])                # x + u >= 0 
constraint_3 = np.hstack([np.zeros((1, 2)), -np.ones((1, 2))])  # -sum(u) >= -1

A = np.vstack([constraint_1, constraint_2, constraint_3])
b = np.hstack([np.zeros(4), [-1]])

In [6]:
print(f"Shape of constraint matrix A: {A.shape}")
print(A)
print(f"Shape of constraint matrix b (RHS): {b.shape}")
print(b)

Shape of constraint matrix A: (5, 4)
[[-1. -0.  1.  0.]
 [-0. -1.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  0.  1.]
 [ 0.  0. -1. -1.]]
Shape of constraint matrix b (RHS): (5,)
[ 0.  0.  0.  0. -1.]


In [7]:
def active_set_qp(G, c, A, b, initial_x_0, tol=1e-3, max_iter=100):
    n = int(G.shape[0] / 2)
    m = int(n / 2)
    
    # Initialize variables
    x = initial_x_0
    W = set(get_initial_W(A, x, b))  # Working set of active constraints
    p_k = None
    
    for iteration in range(max_iter):
        if iteration != 0:
            print("\n")
        print(f"Current iteration: {iteration}")
        print(f"  x_{iteration} = {x.T}")
        print(f"  W_{iteration} = {W}")
        
        # Calculate p_k by solving the subproblem
        
        A_w = np.array([A[i] for i in W]) # Constraint matrix with only the constraints where i in W
        g_k = G @ x + c
        
        '''
        p_k = calculate_pk(G, g_k, A_w)
        
        '''
        if W:
            # Step 1: Find the nullspace of A
            N = null_space(A_w)

            # Step 2: Transform the problem to the nullspace basis
            G_reduced = N.T @ G @ N
            g_k_reduced = N.T @ g_k

            # Step 3: Solve the reduced problem (unconstrained quadratic problem)
            p_reduced = -np.linalg.inv(G_reduced) @ g_k_reduced

            # Step 4: Transform back to the original space
            p_k = N @ p_reduced
            
        else:
            p_k = solve(G + np.eye(n)*0.01, -g_k)
        #'''
            
        print(f"  p_{iteration} = {p_k.T}")
            
        # If p_k == 0, we need to calculate the multipliers
        if np.isclose(0, np.linalg.norm(p_k)): 
            if W: 
                lambda_w = np.linalg.lstsq(A_w.T, g_k, rcond=None)[0]
                print(f"  lambda = {lambda_w.T}")
                
                # Check if all Lagrange multipliers are non-negative
                if all(lambda_w >= -tol): # >= -tol @Metin
                    print("\nFound optimal solution!")
                    print(f"x_opt = {x[:n].T}")
                    return x[:n]  # Optimal solution found
                
                # Identify the most negative Lagrange multiplier
                min_lambda_idx = np.argmin(lambda_w)
                min_lambda_val = list(W)[min_lambda_idx]
                
                # Update the working set by removing the corresponding constraint
                W.remove(min_lambda_val)
            else:
                # shouldn't be able to get here
                print("SHOULDN'T BE ABLE TO GET HERE")
                break
        # if p_k != 0, we need to calculate alpha with a value in the interval [0, 1]
        else: 
            alpha_k, blocking_idx = calc_alpha_k(A, W, b, p_k, x)
            print(f"  alpha_{iteration} = {alpha_k}")
            print(f"  Blocking index = {blocking_idx}")
            
            # new x_k
            x = x + alpha_k * p_k

            if blocking_idx:
                W.add(blocking_idx)
        
    # end of max_iter
    print(f"Max iterations reached! Final x = {x[:n].T}")
    return x[:n]

In [8]:
x_opt = active_set_qp(G_tilde, c_tilde, A, b, z_tilde, max_iter=10)

Current iteration: 0
  x_0 = [[0.1 1.  0.1 1. ]]
  W_0 = {0, 1, 2, 3}
  p_0 = [[0. 0. 0. 0.]]
  lambda = [[ 0.45  0.45 -0.45 -0.45]]


Current iteration: 1
  x_1 = [[0.1 1.  0.1 1. ]]
  W_1 = {0, 1, 3}
  p_1 = [[0.9 0.  0.9 0. ]]
Now inside alpha calculation:
    Constraint 4:
    Nominator b[4] - A[4] @ x = -1.0 - [-1.1] = [0.1]
    Denominator A[4] @ p_k = [-0.9]
  alpha_1 = [-0.11111111]
  Blocking index = 4


Current iteration: 2
  x_2 = [[-8.32667268e-17  1.00000000e+00 -8.32667268e-17  1.00000000e+00]]
  W_2 = {0, 1, 3, 4}
  p_2 = [[0. 0. 0. 0.]]
  lambda = [[ 1.0000000e+00  1.0000000e+00 -8.8817842e-16  1.0000000e+00]]

Found optimal solution!
x_opt = [[-8.32667268e-17  1.00000000e+00]]


In [9]:
#print("Testing if our setup works with the CVXOPT package...")
#print(f"Optimal x: {cvxopt_sol(G_tilde, c_tilde, A, b, 2)}")